In [1]:
import networkx as nx
import igraph as ig
import leidenalg as la
import pandas as pd

In [2]:
graph = "marvel"
graph_file = "_input/triples/" + graph + ".triples"
max_comm_size = 100

In [3]:
graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
G_nx = nx.Graph()
for row in graph_df.itertuples():
    G_nx.add_edge(int(row[1]), int(row[2]))
print(G_nx)

Graph with 262418 nodes and 2030453 edges


In [4]:
node_map = {node: i for i, node in enumerate(G_nx.nodes())}
G = ig.Graph()
G.add_vertices(len(G_nx.nodes()))
G.add_edges([(node_map[u], node_map[v]) for u, v in G_nx.edges()])
G.vs["id"] = list(node_map.keys())

In [5]:
level = 0
communities_file = "./_input/communities/leiden/" + graph + "/level_" + str(level) + ".txt"

partition = la.find_partition(G, la.ModularityVertexPartition, n_iterations=-1, seed=0)

with open(communities_file, "w") as f:
    for community in partition:
        original_ids = [G.vs[node]["id"] for node in community]
        f.write(" ".join(map(str, original_ids)) + "\n")

In [6]:
communities_final = "./_input/communities/leiden/" + graph + "/final.txt"
level = 0
do_next_iteration = True

with open(communities_final, "w") as output_final:

    while do_next_iteration:

        do_next_iteration = False
        level += 1
        communities_read = "./_input/communities/leiden/" + graph + "/level_" + str(level-1) + ".txt"
        communities_file = "./_input/communities/leiden/" + graph + "/level_" + str(level) + ".txt"

        print("##### level: " + str(level) + " #####")

        communities = []
        with open(communities_read, "r") as f:
            for line in f:
                community = list(map(int, line.split()))
                communities.append(community)

        with open(communities_file, "w") as f:
            for community in communities:
                # if len(community) == 158:
                if len(community) > max_comm_size:
                    print("level " + str(level) + ": clustering community with " + str(len(community)) + " nodes")
                    selected_vertex_indices = [v.index for v in G.vs if v["id"] in community]
                    G_sub = G.subgraph(selected_vertex_indices)
                    # print(G_sub.vs["id"])
                    partition = la.find_partition(G_sub, la.ModularityVertexPartition, n_iterations=-1, seed=0)
                    if len(partition) > 1:
                        do_next_iteration = True
                        for comm in partition:
                            original_ids = [G_sub.vs[node]["id"] for node in comm]
                            f.write(" ".join(map(str, original_ids)) + "\n")
                    else:
                        output_final.write(" ".join(map(str, community)) + "\n")
                else:
                    print("level " + str(level) + ": not clustering community with " + str(len(community)) + " nodes")
                    output_final.write(" ".join(map(str, community)) + "\n")

##### level: 1 #####
level 1: clustering community with 26066 nodes
level 1: clustering community with 21111 nodes
level 1: clustering community with 19731 nodes
level 1: clustering community with 18062 nodes
level 1: clustering community with 17653 nodes
level 1: clustering community with 13885 nodes
level 1: clustering community with 10826 nodes
level 1: clustering community with 9433 nodes
level 1: clustering community with 9279 nodes
level 1: clustering community with 8757 nodes
level 1: clustering community with 8366 nodes
level 1: clustering community with 6961 nodes
level 1: clustering community with 6369 nodes
level 1: clustering community with 5905 nodes
level 1: clustering community with 5015 nodes
level 1: clustering community with 4529 nodes
level 1: clustering community with 4271 nodes
level 1: clustering community with 3430 nodes
level 1: clustering community with 3260 nodes
level 1: clustering community with 3153 nodes
level 1: clustering community with 3118 nodes
level 

In [7]:
with open(communities_final, "r") as f:
    sum_len = 0
    for community in f:
        community_string = community.strip().split(" ")
        sum_len += len(community_string)
    print(sum_len)

262418
